# Meneco demo

First you need to install Meneco. For example with `pip` ...

In [ ]:
pip install meneco

     |████████████████████████████████| 5.8MB 3.8MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 40kB 5.4MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
  Created wheel for meneco: filename=Meneco-2.0.0-cp36-none-any.whl size=15271 sha256=d28903c1811afe873239b6ec9b0fcd2138147fa632880785465f3ad895843834
  Stored in directory: /root/.cache/pip/wheels/f2/b1/da/9add58eff34821feb52c528bc86820bf98bc18fc97ed075e27
  Created wheel for clyngor-with-clingo: filename=clyngor_with_clingo-5.3.post1-cp36-none-any.whl size=5798483 sha256=60fdfe7689f7ca04884d291580bc0901bdda5c6beacd3f53dafd0057fb49432b
  Stored in directory: /root/.cache/pip/wheels/38/f3/21/5ec422631f80e8905910c5ed4cabc43740db8404be904a4eb5
  Created wheel for pyPEG2: filename=pyPEG2-2.15.2-cp36-none-any.whl size=22777 sha256=72923dbefd65f82931b17662f0b727ee4e432ca4472bf6e304c3ab8ed5bb9881
  Stored in directory: /root/.cache/pip/wheels/4a/57/58/03557d6f87c87f518b56561f8ee

then you can import the necessary modules ...

In [1]:
from clyngor.as_pyasp import TermSet,Atom
from urllib.request import urlopen
from meneco.meneco import query, utils, sbml

Next, you can load a draft network from an sbml file ...

In [7]:
draft_sbml= urlopen('https://raw.githubusercontent.com/bioasp/meneco/master/Ectodata/ectocyc.sbml')
draftnet = sbml.readSBMLnetwork(draft_sbml, 'draft')
len(draftnet)

8191

load the seeds ...

In [12]:
seeds_sbml = urlopen('https://raw.githubusercontent.com/bioasp/meneco/master/Ectodata/seeds.sbml')
seeds = sbml.readSBMLseeds(seeds_sbml)

and load the targets ...

In [13]:
targets_sbml = urlopen('https://raw.githubusercontent.com/bioasp/meneco/master/Ectodata/targets.sbml')
targets = sbml.readSBMLtargets(targets_sbml)

Then you can check the draft network for unproducible targets ...

In [16]:
model = query.get_unproducible(draftnet, targets, seeds)
unproducible = set(a[0] for pred in model if pred == 'unproducible_target' for a in model[pred])
print('{0} unproducible targets:\n\t{1}\n'.format(len(unproducible), '\n\t'.join(unproducible)))

51 unproducible targets:
	D__45__ALANINE
	_4__45__AMINO__45__BUTYRATE
	VAL
	THREO__45__DS__45__ISO__45__CITRATE
	TYR
	GLYCERATE
	ILE
	GLYCEROL
	THR
	ARACHIDIC_ACID
	L__45__ASPARTATE
	MANNITOL
	HOMO__45__SER
	ARG
	GLY
	CPD__45__8117
	LINOLEIC_ACID
	PALMITATE
	LEU
	GLC
	HIS
	ARACHIDONIC_ACID
	MET
	SUC
	CPD__45__13814
	CPD__45__8120
	GLYCOLLATE
	ASN
	CPD__45__7836
	PHE
	L__45__ALPHA__45__ALANINE
	OLEATE__45__CPD
	GLT
	GLUTATHIONE
	CPD__45__9245
	_5Z8Z11Z14Z17Z__45__EICOSAPENTAENOATE
	CPD__45__14292
	CPD__45__8119
	CPD__45__8121
	CPD__45__9247
	PRO
	LYS
	SER
	GLN
	DOCOSANOATE
	STEARIC_ACID
	TRP
	CIT
	LINOLENIC_ACID
	CYS
	L__45__ORNITHINE



You can load another reaction network like metacyc repair data base ...

In [8]:
repair_sbml = urlopen('https://raw.githubusercontent.com/bioasp/meneco/master/Ectodata/metacyc_16-5.sbml')
repairnet = sbml.readSBMLnetwork(repair_sbml, 'repairnet')
len(repairnet)

64180

and combine the draft network with the repair database ...

In [9]:
combinet = draftnet
combinet = TermSet(combinet.union(repairnet))

In [10]:
len(combinet)

72371

and then check for which targets producibilty cannot be restored even with the combined networks ...

In [14]:
model = query.get_unproducible(combinet, targets, seeds)
never_producible = set(a[0] for pred in model if pred == 'unproducible_target' for a in model[pred])
print('{0} unreconstructable targets:\n\t{1}\n'.format(len(never_producible), '\n\t'.join(never_producible)))

4 unreconstructable targets:
	CPD__45__8119
	DOCOSANOATE
	CPD__45__14292
	CPD__45__13814



and for which targets the production paths are repairable ...

In [17]:
reconstructable_targets = unproducible.difference(never_producible)
print('{0} reconstructable targets:\n\t{1}\n'.format(len(reconstructable_targets), '\n\t'.join(reconstructable_targets)))

47 reconstructable targets:
	D__45__ALANINE
	_4__45__AMINO__45__BUTYRATE
	VAL
	THREO__45__DS__45__ISO__45__CITRATE
	TYR
	GLYCERATE
	ILE
	GLYCEROL
	THR
	ARACHIDIC_ACID
	L__45__ASPARTATE
	MANNITOL
	HOMO__45__SER
	ARG
	GLY
	CPD__45__8117
	LINOLEIC_ACID
	PALMITATE
	LEU
	GLC
	HIS
	ARACHIDONIC_ACID
	MET
	SUC
	CPD__45__8120
	GLYCOLLATE
	ASN
	CPD__45__7836
	PHE
	L__45__ALPHA__45__ALANINE
	OLEATE__45__CPD
	GLT
	GLUTATHIONE
	CPD__45__9245
	_5Z8Z11Z14Z17Z__45__EICOSAPENTAENOATE
	CPD__45__8121
	CPD__45__9247
	PRO
	LYS
	SER
	GLN
	STEARIC_ACID
	TRP
	CIT
	LINOLENIC_ACID
	CYS
	L__45__ORNITHINE



You can compute the essential reactions for the repairable target ...

In [ ]:
essential_reactions = set()
for t in reconstructable_targets:
      single_target = TermSet()
      single_target.add(Atom('target("' + t + '")'))
      print('\nComputing essential reactions for', t,'... ', end=' ')
      model = query.get_intersection_of_completions(draftnet, repairnet, seeds, single_target)
      print(' done.')
      essentials_lst = set(a[0] for pred in model if pred == 'xreaction' for a in model[pred])
      print('{0} essential reactions for target {1}:\n\t{2}'.format(len(essentials_lst), t, '\n\t'.join(essentials_lst)))
      essential_reactions = essential_reactions.union(essentials_lst)
print('Overall {0} essential reactions found:\n\t{1}\n'.format(len(essential_reactions), '\n\t'.join(essential_reactions)))


Computing essential reactions for MET ...   done.
0 essential reactions for target MET:
	

Computing essential reactions for L__45__ORNITHINE ...   done.
0 essential reactions for target L__45__ORNITHINE:
	

Computing essential reactions for ASN ...   done.
0 essential reactions for target ASN:
	

Computing essential reactions for OLEATE__45__CPD ...   done.
5 essential reactions for target OLEATE__45__CPD:
	RXN__45__9520
	_1__46__14__46__19__46__2__45__RXN
	RXN__45__9655
	_3__46__1__46__2__46__14__45__RXN
	_4__46__2__46__1__46__59__45__RXN

Computing essential reactions for CPD__45__7836 ...   done.
6 essential reactions for target CPD__45__7836:
	RXN__45__9520
	RXN__45__9533
	RXN__45__9655
	RXN__45__10727
	RXN__45__9537
	_4__46__2__46__1__46__59__45__RXN

Computing essential reactions for ILE ...   done.
0 essential reactions for target ILE:
	

Computing essential reactions for CPD__45__8117 ...   done.
5 essential reactions for target CPD__45__8117:
	RXN__45__9520
	_1__46__14__46__

You can compute a completion of minimal size suitable to produce all reconstructable targets ...

In [18]:
targets = TermSet(Atom('target("' + t+'")') for t in reconstructable_targets)
model =  query.get_minimal_completion_size(draftnet, repairnet, seeds, targets)
one_min_sol_lst = set(a[0] for pred in model if pred == 'xreaction' for a in model[pred])
optimum = len(one_min_sol_lst)

print('minimal size =',optimum)

print('One minimal completion of size {0}:\n\t{1}\n'.format(
            optimum, '\n\t'.join(one_min_sol_lst)))

KeyboardInterrupt: 

We can compute the common reactions in all completion with a given size ...

In [ ]:
model = query.get_intersection_of_optimal_completions(draftnet, repairnet, seeds, targets,  optimum)
cautious_sol_lst = set(a[0] for pred in model if pred == 'xreaction' for a in model[pred])

print('Intersection of all solutions of size {0}:\n\t{1}\n'.format(
            optimum, '\n\t'.join(cautious_sol_lst)))

Intersection of all solutions of size 45:
	RXN__45__9549
	RXN__45__8346
	_4__46__2__46__1__46__61__45__RXN
	RXN__45__9533
	RXN__45__9655
	GLUTDECARBOX__45__RXN
	RXN__45__13435
	RXN__45__12777
	MANNITOL__45__1__45__PHOSPHATASE__45__RXN
	_4__46__2__46__1__46__59__45__RXN
	HISTCYCLOHYD__45__RXN
	_4__46__2__46__1__46__58__45__RXN
	RXN__45__10727
	RXN__45__9634
	RXN__45__12968
	RXN__45__13261
	RXN__45__13242
	RXN__45__12969
	RXN__45__9673
	RXN__45__9550
	RXN__45__9548
	RXN__45__9558
	RXN__45__9557
	RXN__45__9520
	_1__46__14__46__19__46__2__45__RXN
	HISTIDPHOS__45__RXN
	RXN__45__12971
	_3__46__1__46__2__46__14__45__RXN
	RXN__45__9537
	RXN__45__12766
	RXN__45__9555



We can compute the **union** of all completion with a given size ...

In [ ]:
model = query.get_union_of_optimal_completions(draftnet, repairnet, seeds, targets, optimum)
brave_sol_lst = set(a[0] for pred in model if pred == 'xreaction' for a in model[pred])

print('Intersection of all solutions of size {0}:\n\t{1}\n'.format(
            optimum, '\n\t'.join(brave_sol_lst)))

Intersection of all solutions of size 45:
	RXN__45__8349
	RXN__45__9549
	RXN__45__8346
	_4__46__2__46__1__46__61__45__RXN
	ADENOSYLHOMOCYSTEINASE__45__RXN
	RXN__45__12794
	_3__46__4__46__13__46__19__45__RXN
	RXN__45__9533
	RXN__45__13431
	RXN__45__9655
	_4__46__3__46__1__46__25__45__RXN
	GLUTDECARBOX__45__RXN
	PREPHENATE__45__ASP__45__TRANSAMINE__45__RXN
	RXN__45__12755
	RXN__45__13435
	RXN__45__12777
	_3__46__4__46__13__46__17__45__RXN
	MANNITOL__45__1__45__PHOSPHATASE__45__RXN
	_4__46__2__46__1__46__59__45__RXN
	HISTCYCLOHYD__45__RXN
	_4__46__2__46__1__46__58__45__RXN
	RXN__45__8389
	PHOSPHOGLYCERATE__45__PHOSPHATASE__45__RXN
	RXN__45__7968
	_3__46__4__46__13__46__4__45__RXN
	RXN__45__9548
	_4__46__3__46__1__46__23__45__RXN
	ACP__45__S__45__ACETYLTRANSFER__45__RXN
	PREPHENATEDEHYDROG__45__RXN
	RXN__45__10727
	RXN__45__11243
	RXN__45__9634
	TRANS__45__CINNAMATE__45__4__45__MONOOXYGENASE__45__RXN
	ADENYLYLSULFATE__45__REDUCTASE__45__RXN
	TYRAMINOTRANS__45__RXN
	RXN__45__12968
	RXN__45_

And finally compute **all** (for this notebook we print only the first three) completions with a given size ...


In [ ]:
models = query.get_optimal_completions(draftnet, repairnet, seeds, targets, optimum)
count = 0
for model in models:
   one_min_sol_lst = set(a[0] for pred in model if pred == 'xreaction' for a in model[pred])
   count += 1
   print('Completion {0}:\n\t{1}\n'.format(
            str(count), '\n\t'.join(one_min_sol_lst)))
   if count == 3:  break

Completion 1:
	RXN__45__8349
	RXN__45__9549
	RXN__45__8346
	_4__46__2__46__1__46__61__45__RXN
	RXN__45__9533
	ADENOSYLHOMOCYSTEINASE__45__RXN
	RXN__45__13431
	RXN__45__9655
	GLUTDECARBOX__45__RXN
	RXN__45__13435
	RXN__45__12777
	MANNITOL__45__1__45__PHOSPHATASE__45__RXN
	_4__46__2__46__1__46__59__45__RXN
	HISTCYCLOHYD__45__RXN
	_4__46__2__46__1__46__58__45__RXN
	PHOSPHOGLYCERATE__45__PHOSPHATASE__45__RXN
	RXN__45__7968
	RXN__45__10727
	RXN__45__11243
	RXN__45__9634
	ADENYLYLSULFATE__45__REDUCTASE__45__RXN
	RXN__45__12968
	RXN__45__13261
	RXN__45__12361
	RXN__45__13242
	RXN__45__12969
	RXN__45__9673
	RXN__45__10663
	RXN__45__9550
	RXN__45__9548
	RXN__45__9558
	RXN__45__9555
	RXN__45__9557
	_4__45__COUMARATE__45____45__COA__45__LIGASE__45__RXN
	PHENYLALANINE__45__AMMONIA__45__LYASE__45__RXN
	RXN__45__9520
	_1__46__14__46__19__46__2__45__RXN
	HISTIDPHOS__45__RXN
	RXN__45__12971
	RXN__45__9697
	RXN__45__8347
	RXN__45__9537
	_3__46__1__46__2__46__14__45__RXN
	RXN__45__12766
	TRANS__45__CINN

That's all folks!